In [3]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.db_conn import DB_CONN_JJTG_DATA
from quant_utils.performance import Performance

In [4]:
def get_barra_exposure(trade_dt):
    query_sql = f"""
    SELECT 
        * 
    FROM 
        dy1d_exposure_cne6_sw21 
    WHERE 
        TRADE_DT = ( 
            SELECT max( TRADE_DT ) 
            FROM dy1d_exposure_cne6_sw21 
            WHERE TRADE_DT <= '{trade_dt}' 
        )
    """
    result = DB_CONN_JJTG_DATA.exec_query(query_sql)
    result = (
        result
        .drop(columns=["ID", "UPDATE_TIME", "TRADE_DT"])
    )
    return result.dropna(axis=1)

def get_fund_holdings(report_date):
    query_sql = f"""
    SELECT
        a.FUND_ID,
        a.RATIO_IN_NA,
        a.RATIO_IN_EQUITY,
        a.HOLDING_TICKER_SYMBOL as TICKER_SYMBOL
    FROM
        fund_holdings a
    WHERE
        1 = 1 
        AND a.REPORT_DATE = '{report_date}'
    """
    return DB_CONN_JJTG_DATA.exec_query(query_sql)


In [5]:
def get_fund_barra_exposure(report_date):
    fund_holding_df = get_fund_holdings(report_date=report_date)
    barra_exposure = get_barra_exposure(report_date)
    temp_df = fund_holding_df.merge(barra_exposure)
    print(temp_df)
    def _cal_fund_barra_exposure(grouped_df):
        temp_weight_df = grouped_df[["RATIO_IN_NA", "RATIO_IN_EQUITY"]]
        temp_barra_df = grouped_df[grouped_df.columns[list(grouped_df.columns).index("BETA"):]]
        return temp_weight_df.T.dot(temp_barra_df)
    result = (
        temp_df.groupby(by="FUND_ID")
        .apply(_cal_fund_barra_exposure)
        .reset_index()
        .rename(columns={"level_1": "RATIO_TYPE"})
    )
    result["REPORT_DATE"] = report_date
    return result


In [6]:
report_date = '20231231'
fund_holding_df = get_fund_holdings(report_date=report_date)
barra_exposure = get_barra_exposure(report_date)
temp_df = fund_holding_df.merge(barra_exposure)

In [7]:
def _cal_fund_barra_exposure(grouped_df):
    temp_weight_df = grouped_df[["RATIO_IN_NA", "RATIO_IN_EQUITY"]]
    temp_barra_df = grouped_df[grouped_df.columns[list(grouped_df.columns).index("BETA"):]]
    return temp_weight_df.T.dot(temp_barra_df)
result = (
    temp_df.groupby(by="FUND_ID")
    .apply(_cal_fund_barra_exposure)
    .reset_index()
    .rename(columns={"level_1": "RATIO_TYPE"})
)
result["REPORT_DATE"] = report_date
DB_CONN_JJTG_DATA.upsert(result, "fund_derivatives_holding_barra_exposure")

写入192.20.57.208:chentiancheng-fund_derivatives_holding_barra_exposure受影响行数为13926行!


写入192.20.57.208:chentiancheng--fund_derivatives_holding_barra_exposure-受影响行数为0行!


In [ ]:
get_barra_exposure("20081231").columns

Index(['TICKER_SYMBOL', 'BETA', 'MOMENTUM', 'SIZE', 'EARNYILD', 'RESVOL',
       'GROWTH', 'BTOP', 'LEVERAGE', 'LIQUIDTY', 'MIDCAP', 'DIVYILD',
       'EARNQLTY', 'EARNVAR', 'INVSQLTY', 'LTREVRSL', 'PROFIT', 'ANALSENTI',
       'INDMOM', 'SEASON', 'STREVRSL', 'Bank', 'RealEstate', 'Health',
       'Transportation', 'Mining', 'NonFerMetal', 'HouseApp', 'LeiService',
       'MachiEquip', 'BuildDeco', 'CommeTrade', 'CONMAT', 'Auto', 'Textile',
       'FoodBever', 'Electronics', 'Computer', 'LightIndus', 'Utilities',
       'Telecom', 'AgriForest', 'CHEM', 'Media', 'IronSteel', 'NonBankFinan',
       'ELECEQP', 'AERODEF', 'Conglomerates', 'COUNTRY'],
      dtype='object')

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, paired_distances
query_sql = """
SELECT
	b.TICKER_SYMBOL,
	b.SEC_SHORT_NAME,
	c.LEVEL_1,
	c.LEVEL_2,
	c.LEVEL_3,
	a.* 
FROM
	fund_derivatives_holding_barra_exposure a
	JOIN fund_info b ON a.FUND_ID = b.FUND_ID
	JOIN fund_type_own c ON c.REPORT_DATE = a.REPORT_DATE 
	AND c.TICKER_SYMBOL = b.TICKER_SYMBOL 
WHERE
	1 = 1 
	AND a.REPORT_DATE = '20221231' 
	AND b.IS_MAIN = 1 
	AND level_1 = "固收+" 
	AND a.RATIO_TYPE = 'RATIO_IN_NA'
	and b.IS_ILLIQUID = 0
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql).dropna(axis=1)
temp = df[df.columns[list(df.columns).index("BETA"): list(df.columns).index("COUNTRY")+1]]

In [ ]:
result = pd.DataFrame(cosine_similarity(temp.values, temp.values))
result.index = df["SEC_SHORT_NAME"].tolist()
result.columns = df["SEC_SHORT_NAME"].tolist()

In [ ]:
df1 = result.unstack().reset_index()
df1 = df1[df1[0] < 0.9999 ]
df1["持仓风格相似度_rank"] = df1.groupby(by=["level_0"])[0].rank(ascending=False, pct=True)
df1 = df1.rename(columns={"level_0": "主基金", "level_1": "备选基金", 0: "持仓风格相似度"})

In [ ]:
query_sql = """
SELECT
a.END_DATE,
	a.TICKER_SYMBOL,
	a.LEVEL_1,
	a.LEVEL_2,
	a.LEVEL_3,
	a.LOG_ALPHA_LEVEL_1,
	a.LOG_ALPHA_LEVEL_2,
	a.LOG_ALPHA_LEVEL_3 
FROM
	fund_derivatives_fund_log_alpha a

WHERE
	1 = 1 
	AND ( a.END_DATE BETWEEN '2022-08-15' AND '2023-08-15' ) 
	AND a.LEVEL_1 = "固收+"
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
query_sql = """
	SELECT
		TICKER_SYMBOL,
		SEC_SHORT_NAME
	FROM
		fund_info b 
	WHERE
		1 = 1 
		AND b.IS_MAIN = 1 
		AND b.IS_ILLIQUID = 0 
		AND b.ESTABLISH_DATE <= '2022-02-15' AND IFNULL( b.EXPIRE_DATE, '20991231' ) >= '2023-08-15' 
"""
fund_info = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [ ]:
df = df.merge(fund_info)

In [ ]:
df = df.pivot_table(index="END_DATE", columns="SEC_SHORT_NAME",values="LOG_ALPHA_LEVEL_2")
corr_df = df.corr()
df2 = corr_df.unstack()
df2.index.names = ["主基金", "备选基金"] 
df2 = df2.reset_index().rename(columns={0:"超额收益相似度"})


In [ ]:
df2 = df2[df2["超额收益相似度"] !=1 ]
df2["超额收益相似度_rank"] = df2.groupby(by=["主基金"])["超额收益相似度"].rank(ascending=False, pct=True)

In [ ]:
df2["超额收益相似度_rank"] = df2.groupby(by=["主基金"])["超额收益相似度"].rank(ascending=False, pct=True)

In [ ]:
result = df1.merge(df2, how="inner")
result["总相似度"] = (1-result["持仓风格相似度_rank"])*0.3 + (1-result["超额收益相似度_rank"])*0.7

In [ ]:
result["rank"] = result.groupby(by="主基金")["总相似度"].rank(ascending=False)

In [ ]:
result = result[(result["rank"]<=10)| (result["rank"]>=(len(result["主基金"].unique())-10))]

In [ ]:
result.to_excel("d:/超额收益相似性-固收+.xlsx")

In [ ]:
file_path = "d:/产品对比.xlsx"
df = pd.read_excel(file_path).set_index("END_DATE")

In [ ]:
df.corr().to_excel("d:/相关性.xlsx")

In [ ]:
(df/100+1).cumprod().to_excel("d:/净值.xlsx")